In [1]:
import os

import torch
import torch.nn.utils.prune as prune
from transformers import DetrImageProcessor, DetrForObjectDetection

import utils

In [2]:
image_sets_path = '/mnt/data/DatasetsML/CV/VOC_2012/VOC2012/ImageSets/Main/'
train_path = os.path.join(image_sets_path, 'train.txt')
val_path = os.path.join(image_sets_path, 'val.txt')

images_path = '/mnt/data/DatasetsML/CV/VOC_2012/VOC2012/JPEGImages/'
annnots_path = '/mnt/data/DatasetsML/CV/VOC_2012/VOC2012/Annotations/'

train_images = utils.read_set(train_path)
val_images = utils.read_set(val_path)

In [3]:
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")

The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.
Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a Ber

In [4]:
print(f"Model size: {utils.model_size(model):.3f} MB")

Model size: 158.404 MB


# Random Unstructured

In [11]:
model_rndm_unstr = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")

Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
for name, module in model_rndm_unstr.named_modules():
    if isinstance(module, torch.nn.Conv2d):
        module = prune.random_unstructured(
            module, name='weight', amount=0.05
        )

In [13]:
print(f"Model size: {utils.model_size(model_rndm_unstr):.3f} MB")

Model size: 158.404 MB


In [14]:
rndm_unstr_map, rndm_unstr_sec_per_frame = utils.evaluate_over_voc(
    images_path=images_path,
    annots_path=annnots_path,
    val_images=val_images,
    model=model_rndm_unstr,
    proc=processor,
)

  0%|          | 0/5823 [00:00<?, ?it/s]

In [15]:
rndm_unstr_map

{'map': tensor(0.3023),
 'map_50': tensor(0.4269),
 'map_75': tensor(0.3252),
 'map_small': tensor(0.0889),
 'map_medium': tensor(0.2287),
 'map_large': tensor(0.3514),
 'mar_1': tensor(0.2754),
 'mar_10': tensor(0.3533),
 'mar_100': tensor(0.3543),
 'mar_small': tensor(0.1085),
 'mar_medium': tensor(0.2710),
 'mar_large': tensor(0.3984),
 'map_per_class': tensor(-1.),
 'mar_100_per_class': tensor(-1.),
 'classes': tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19], dtype=torch.int32)}

In [16]:
rndm_unstr_sec_per_frame

0.087

# L1 Unstructured

In [19]:
model_l1_unstr = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")

Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
for name, module in model_l1_unstr.named_modules():
    if isinstance(module, torch.nn.Conv2d):
        prune.l1_unstructured(module, name='weight', amount=0.05)
    elif isinstance(module, torch.nn.Linear):
        prune.l1_unstructured(module, name='weight', amount=0.05)

In [21]:
print(f"Model size: {utils.model_size(model_l1_unstr):.3f} MB")

Model size: 158.404 MB


In [23]:
rndm_l1_map, rndm_l1_sec_per_frame = utils.evaluate_over_voc(
    images_path=images_path,
    annots_path=annnots_path,
    val_images=val_images,
    model=model_l1_unstr,
    proc=processor,
)

  0%|          | 0/5823 [00:00<?, ?it/s]

In [24]:
rndm_l1_map

{'map': tensor(0.5607),
 'map_50': tensor(0.7275),
 'map_75': tensor(0.6069),
 'map_small': tensor(0.1946),
 'map_medium': tensor(0.4117),
 'map_large': tensor(0.6605),
 'mar_1': tensor(0.4638),
 'mar_10': tensor(0.6239),
 'mar_100': tensor(0.6262),
 'mar_small': tensor(0.2536),
 'mar_medium': tensor(0.4804),
 'mar_large': tensor(0.7113),
 'map_per_class': tensor(-1.),
 'mar_100_per_class': tensor(-1.),
 'classes': tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19], dtype=torch.int32)}

In [25]:
rndm_l1_sec_per_frame

0.089

# Ln Unstructured

In [4]:
model_ln_unstr = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")

Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
for name, module in model_ln_unstr.named_modules():
    if isinstance(module, torch.nn.Conv2d):
        prune.ln_structured(module, name='weight', amount=0.05, n=2, dim=1)
    elif isinstance(module, torch.nn.Linear):
        prune.ln_structured(module, name='weight', amount=0.05, n=2, dim=0)

In [6]:
print(f"Model size: {utils.model_size(model_ln_unstr):.3f} MB")

Model size: 158.404 MB


In [7]:
rndm_ln_map, rndm_ln_sec_per_frame = utils.evaluate_over_voc(
    images_path=images_path,
    annots_path=annnots_path,
    val_images=val_images,
    model=model_ln_unstr,
    proc=processor,
)

  0%|          | 0/5823 [00:00<?, ?it/s]

In [8]:
rndm_ln_map

{'map': tensor(0.1291),
 'map_50': tensor(0.2739),
 'map_75': tensor(0.1131),
 'map_small': tensor(0.0018),
 'map_medium': tensor(0.0267),
 'map_large': tensor(0.1943),
 'mar_1': tensor(0.1955),
 'mar_10': tensor(0.2822),
 'mar_100': tensor(0.3188),
 'mar_small': tensor(0.0166),
 'mar_medium': tensor(0.1179),
 'mar_large': tensor(0.4290),
 'map_per_class': tensor(-1.),
 'mar_100_per_class': tensor(-1.),
 'classes': tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19], dtype=torch.int32)}

In [9]:
rndm_ln_sec_per_frame

0.09